In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rand

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [4]:
import os
os.chdir("C:\\code\\activ")

In [5]:
df = pd.read_excel('./documents/feature.xlsx')

In [6]:
df

,Unnamed: 0,합계출산율,국내 총인구,서울인구,경기도인구,인천시인구,수도권 인구,신생아 출산인구,사망인구,혼인건수,...,- 농축수산물,- 공업제품,- 집세,- 공공서비스,- 개인서비스,소비자 물가 지수,금리,원 달러 환율,"국제유가(WTI, 달러)",국내총생산
0,1986-01-01,1.58,41265113,9798542,5075449,1441131,16315122,69708,22924,36162,...,5.4,2.3,4.7,2.3,4.5,30.117,9.70,890.53,19.66,102985.8
1,1986-02-01,1.58,41265113,9798542,5075449,1441131,16315122,73659,20258,36522,...,5.3,2.2,4.5,2.3,4.5,30.293,9.70,886.39,13.23,102985.8
2,1986-03-01,1.58,41265113,9798542,5075449,1441131,16315122,48593,22144,38380,...,5.8,2.2,4.5,2.1,5.2,30.353,9.70,884.37,10.40,102985.8
3,1986-04-01,1.58,41265113,9798542,5075449,1441131,16315122,47880,20892,36333,...,5.4,1.3,4.3,2.6,5.1,30.323,9.70,885.60,13.34,102985.8
4,1986-05-01,1.58,41265113,9798542,5075449,1441131,16315122,47800,18495,37520,...,4.7,1.7,4.0,2.4,5.1,30.501,9.70,887.21,14.25,102985.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,2021-07-01,0.81,51671569,9558153,13512867,2937440,26008460,22356,25748,15739,...,11.7,1.1,1.1,0.3,2.3,102.260,0.50,1143.98,73.95,2057448.0
427,2021-08-01,0.81,51669716,9550227,13530519,2938429,26019175,22287,25953,14720,...,10.9,1.3,1.2,0.4,2.3,102.750,0.75,1160.34,68.50,2057448.0
428,2021-09-01,0.81,51667688,9542256,13542284,2941795,26026335,21893,25674,13733,...,10.0,1.5,1.2,0.4,2.4,103.170,0.75,1169.54,75.03,2057448.0
429,2021-10-01,0.81,51662290,9532428,13549577,2945009,26027014,20741,27775,15203,...,8.9,1.8,1.3,0.9,2.4,103.350,0.75,1182.82,83.57,2057448.0


In [11]:
def make_dataset(training_data,k,test_size=0.2):
    scaler = MinMaxScaler()
    training_data = scaler.fit_transform(training_data.to_numpy()[:,1::])
    p = []
    for i in range(k):
        idx = rand.randint(0, 100)
        X = training_data[idx:idx+100].reshape(100, 1, 21)
        y = training_data[idx+1:idx+1+100]
        X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=test_size, shuffle=False)
        p.append([X_train,X_test,y_train,y_test])

    return p
p = make_dataset(df, 5)

In [8]:
class my_model(tf.keras.Model):
    def __init__(self):
        super(my_model, self).__init__()
        self.d1 = tf.keras.layers.LSTM(units=128, activation='tanh', return_sequences=True)
        self.d2 = tf.keras.layers.LSTM(units=128, activation='tanh', return_sequences=True)
        self.d3 = tf.keras.layers.LSTM(units=128, activation='tanh', return_sequences=False)
        self.d4 = tf.keras.layers.Dense(units=1, activation='linear')
    def call(self, inputs):
        x = self.d1(inputs)
        x = self.d2(x)
        x = self.d3(x)
        x = self.d4(x)

        return x

In [9]:
model_1 = my_model()
model_2 = my_model()
model_3 = my_model()
model_4 = my_model()
model_5 = my_model()

In [10]:
model_list = [model_1,model_2,model_3,model_4,model_5]

In [13]:
for i in range(5):
    model_list[i].build(input_shape=(1, 1, 21))
    model_list[i].compile(optimizer='adam', loss='mse')

In [14]:

for i, model in enumerate(model_list):
    X_train, X_test, y_train, y_test = p[i][0], p[i][1], p[i][2], p[i][3]
    hist = model.fit(X_train, y_train, epochs=5000, batch_size=64, validation_data=(X_test, y_test), verbose=1, 
                callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)])

Epoch 1/5000
2/2 [==============================] - 13s 2s/step - loss: 0.2465 - val_loss: 0.2469
Epoch 2/5000
2/2 [==============================] - 0s 27ms/step - loss: 0.2308 - val_loss: 0.2271
Epoch 3/5000
2/2 [==============================] - 0s 65ms/step - loss: 0.2140 - val_loss: 0.2032
Epoch 4/5000
2/2 [==============================] - 0s 28ms/step - loss: 0.1933 - val_loss: 0.1739
Epoch 5/5000
2/2 [==============================] - 0s 31ms/step - loss: 0.1687 - val_loss: 0.1395
Epoch 6/5000
2/2 [==============================] - 0s 29ms/step - loss: 0.1408 - val_loss: 0.1025
Epoch 7/5000
2/2 [==============================] - 0s 67ms/step - loss: 0.1119 - val_loss: 0.0713
Epoch 8/5000
2/2 [==============================] - 0s 39ms/step - loss: 0.0901 - val_loss: 0.0625
Epoch 9/5000
2/2 [==============================] - 0s 36ms/step - loss: 0.0892 - val_loss: 0.0783
Epoch 10/5000
2/2 [==============================] - 0s 32ms/step - loss: 0.1042 - val_loss: 0.0786
Epoch 11/5

In [22]:
for i in range(5):
    model_list[i].save('./model/action_net{0}'.format(i))

INFO:tensorflow:Assets written to: ./model/action_net0\assets


INFO:tensorflow:Assets written to: ./model/action_net0\assets


INFO:tensorflow:Assets written to: ./model/action_net1\assets


INFO:tensorflow:Assets written to: ./model/action_net1\assets


INFO:tensorflow:Assets written to: ./model/action_net2\assets


INFO:tensorflow:Assets written to: ./model/action_net2\assets


INFO:tensorflow:Assets written to: ./model/action_net3\assets


INFO:tensorflow:Assets written to: ./model/action_net3\assets


INFO:tensorflow:Assets written to: ./model/action_net4\assets


INFO:tensorflow:Assets written to: ./model/action_net4\assets
